In [16]:
# Import findspark and initialize. 
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.applications import VGG16, InceptionV3
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, DoubleType
import pandas as pd
import findspark
findspark.init()

In [3]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [4]:
face_recog = spark.read.csv('Resources/Emotion_dataset/data.csv', header=True)
face_recog = face_recog.drop('_c0')

face_recog.printSchema()
face_recog.show(5)


root
 |-- path: string (nullable = true)
 |-- label: string (nullable = true)

+--------------------+--------+
|                path|   label|
+--------------------+--------+
|Surprise/1bd930d6...|Surprise|
|Surprise/cropped_...|Surprise|
|Surprise/0df0e470...|Surprise|
|Surprise/cropped_...|Surprise|
|Surprise/cropped_...|Surprise|
+--------------------+--------+
only showing top 5 rows



In [11]:
base_model = InceptionV3(weights='imagenet', include_top=False)

# Define a function to read and preprocess image data
def read_and_preprocess_image(image_path):
    # Read image from file
    image = cv2.imread(image_path)
    # Preprocess image (resize, normalize, etc.)
    resized_image = image_size(image, target)
    img_arr = img_array(resized_image)
    norm_img = normalize_img(img_arr)
    return norm_img

# Register UDF for reading and preprocessing image data
read_and_preprocess_image_udf = udf(read_and_preprocess_image, ArrayType(DoubleType()))

# Apply UDF to DataFrame to preprocess image data
preprocessed_data = face_recog.withColumn("preprocessed_image", read_and_preprocess_image_udf(face_recog["path"]))

# Define a function to extract features from preprocessed images
def extract_features(image):
    # Resize image to match the input size expected by the model
    resized_image = tf.image.resize(image, (222, 222))
    # Preprocess image for the InceptionV3 model
    preprocessed_image = tf.keras.applications.inception_v3.preprocess_input(resized_image)
    # Extract features from the pre-trained model
    features = base_model.predict(tf.expand_dims(preprocessed_image, axis=0))
    # Reshape features to 1D array
    features = features.flatten()
    return features

# Register UDF for feature extraction
extract_features_udf = udf(extract_features, ArrayType(DoubleType()))

# Apply UDF to DataFrame to extract features
feature_data = preprocessed_data.withColumn("features", extract_features_udf(preprocessed_data["preprocessed_image"]))




DataFrame[path: string, label: string, preprocessed_image: array<double>, features: array<double>]

In [17]:
feature_data_dumb = pd.get_dummies(feature_data)
feature_data_dumb.head()

,"DataFrame[path: string, label: string, preprocessed_image: array<double>, features: array<double>]"
0,True


In [18]:
target = 'label'
features = feature_data.drop(columns=[target])
target_values = feature_data[target]
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(features, target_values, test_size=0.2, random_state=42)

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)



TypeError: DataFrame.drop() got an unexpected keyword argument 'columns'